This notebook implement the ATM machine.


In [1]:
class Card:
    def __init__(self, cardID, pin, accounts=None):
        self.cardID = cardID
        self.pin = pin
        
        if accounts is not None:
            accountIDs = [account.accountID for account in accounts]
            if len(accountIDs) != len(set(accountIDs)):
                raise ValueError("Account numbers should not be overlapped.")

        self.accounts = accounts or []

    def add_account(self, account):
        assert isinstance(account, Account), "Implementation error."

        accountIDs = [account.accountID for account in self.accounts]
        if account.accountID in accountIDs:
            print("The account number is overlapped. Use other account number.")
        else:
            self.accounts.append(account)
    
    def select_account(self, idx):
        return self.accounts[idx]


class Account:
    def __init__(self, accountID, balance=0):
        self.accountID = accountID
        self.balance = balance
    
    def see_balance(self):
        return self.balance
    
    def deposit(self, money):
        self.balance += money
    
    def withdraw(self, money):
        self.balance -= money


class ATM:
    def __init__(self, language=0):
        """
        language 0 : English
        language 1 : Korean
        """
        if language == 0:
            print("English language is chosen.")
        elif language == 1:
            print("한국어를 선택하셨습니다.")
        else:
            raise ValueError("English and Korean are the only supported languages. 영어와 한국어만 지원됩니다.")

        self.language = language
        self.black_list = []
        
        self.inserted_card = None
        self.viewing_account = None
        self.allow_access = False
        self.err_cnt = 0

    def insert_card(self, card):
        is_valid = True
        if self.inserted_card is None:
            if card.cardID not in self.black_list:
                self.inserted_card = card
                if self.language:
                    print("기기에 카드가 삽입되었습니다.")
                else:
                    print("A card is inserted to the machine")
            else:
                if self.language:
                    print("사용이 불가능한 카드입니다. 다른 카드를 사용해주세요.")
                else:
                    print("This card is no more useable. Please use other cards.")
                self.eject_card()
                is_valid = False

        else:
            if self.language:
                print("기기에 이미 카드가 존재합니다. 새로운 카드를 삽입하시려면, 해당 카드를 먼저 제거해주세요.")
            else:
                print("Card is already inserted. You should remove the inserted card first.")

        return is_valid

    def eject_card(self):
        self.inserted_card = None
        self.viewing_account = None
        self.allow_access = False
        self.err_cnt = 0
        
        if self.language:
            print("기기로부터 카드를 반출합니다.")
        else:
            print("A card is ejected from the machine.")

    def check_pin(self, pin):
        assert self.inserted_card is not None, "Implementation error."

        if pin == self.inserted_card.pin:
            if self.language:
                print("올바른 비밀번호를 입력했습니다. 이제 카드내 계좌에 접근 가능합니다.")
            else:
                print("You have entered the correct password. You can now access your account on the card.")
            
            is_done = True
            self.allow_access = True
        else:
            is_done = False
            self.err_cnt += 1

            if self.language:
                print(f"비밀번호 오류입니다. 5회 오류 시 카드 사용이 제한됩니다. 현재 : {self.err_cnt}회.")
            else:
                print(f"Wrong password. Card is no more useable when having 5th errors. Current : {self.err_cnt}th error(s).")
            
            if self.err_cnt == 5:
                self.black_list.append(self.inserted_card.cardID)
                if self.language:
                    print("오류 횟수가 5회에 도달하여 기기에서 카드가 반출됩니다. 해당 카드는 더이상 사용하실 수 없습니다.")
                else:
                    print("The card is ejected from the machine as the number of errors reaches 5. This card can no longer be used.")
                self.eject_card()
                is_done = True

        return is_done

    def select_account(self, index):
        assert self.inserted_card, "Implementation error."
        
        if not self.allow_access:
            if self.language:
                print("계좌를 확인하기 전에 PIN을 먼저 입력해주세요.")
            else:
                print("Please enter your PIN first before verifying your account.")
        else:
            num_accounts = len(self.inserted_card.accounts)
            if index > num_accounts or index < 1:
                if self.language:
                    print("총 계좌의 개수보다 크거나 1보다 작은 숫자를 입력하셨습니다. 올바른 숫자를 입력해주세요.")
                else:
                    print("You entered a number greater than the total number of accounts or less than 1. Please enter the correct number.")

            else:
                if self.viewing_account is None:
                    if self.language:
                        print(f"{index}번째 계좌가 선택되었습니다.")
                    else:
                        print(f"Selected the {index}th account.")
                else:
                    if index == self.index:
                        if self.language:
                            print("이미 해당 계좌를 조회 중입니다.")
                        else:
                            print("The account is already being viewed.")
                    else:
                        if self.language:
                            print(f"{index}번째 계좌로 전환되었습니다.")
                        else:
                            print(f"Switched to the {index}th account.")

                self.viewing_account = self.inserted_card.select_account(index-1)
                self.index = index

    def do_request(self, task, money=None):
        """
        task 0 : see balance
        task 1 : deposit
        task 2 : withdraw
        """
        assert self.inserted_card and self.allow_access and self.viewing_account, "Implementation error."

        if task == 0:
            balance = self.viewing_account.see_balance()
            if self.language:
                print(f"{self.index}번째 계좌에 {balance}원이 예치되어있습니다.")
            else:
                print(f"{balance} won is deposited in the {self.index}th account.")
        else:
            assert money is not None, "금액은 숫자로 입력해야 합니다."

            if task == 1:            
                self.viewing_account.deposit(money)
                
                upd_balance = self.viewing_account.see_balance()
                if self.language:
                    print(f"{self.index}번째 계좌에 {money}원을 추가로 예치하였습니다.")
                    print(f"{self.index}번째 계좌 잔액은 {upd_balance}원입니다.")
                else:
                    print(f"{money} won is added to the {self.index}th account.")
                    print(f"The balance of the {self.index}th account is {upd_balance}.")
                
            elif task == 2:
                balance = self.viewing_account.see_balance()
                if money > balance:
                    if self.language:
                        print("계좌내 금액보다 더 큰 금액을 인출할 수 없습니다.")
                    else:
                        print("You cannot withdraw more than the amount in the account")
                else:
                    self.viewing_account.withdraw(money)

                    upd_balance = self.viewing_account.see_balance()
                    if self.language:
                        print(f"{self.index}번째 계좌에서 {money}원을 인출하였습니다.")
                        print(f"{self.index}번째 계좌 잔액은 {upd_balance}원입니다.")
                    else:
                        print(f"{money} won is withdrawed from the {self.index}th account.")
                        print(f"The balance of the {self.index}th account is {upd_balance}.")


In [2]:
def create_atm():
    language = input("Let's create a new atm \n" +
                     "Enter 0 or 1 to choose a language for instruction by atm: 0 for English, 1 for Korean \n")
    language = int(language)
    atm_machine = ATM(language)
    return language, atm_machine

def create_card():
    card_input = input("Let's create a new card \n" +
                       "Enter two digits with ',' between: 1st for cardID and 2nd for pin. ex) 1, 2 \n ")
    cardID, pin = card_input.split(",")
    cardID, pin = cardID.strip(" "), pin.strip(" ")
    card = Card(cardID, pin)
    return card

def create_account():
    account_input = input("Let's create a new account \n" +
                          "Enter two digits with ',' between: 1st for accountID and 2nd for balance. ex) 3, 4 \n ")
    accountID, balance = account_input.split(",")
    accountID, balance = accountID.strip(" "), int(balance)
    account = Account(accountID, balance)
    return account
    
def main(num_account):
    language, atm_machine = create_atm()

    while True:
        print("-----------------------------------------------------------------------\n")
        card = create_card()
        print("-----------------------------------------------------------------------\n")

        for _ in range(num_account):
            account = create_account()
            card.add_account(account)

        alphabets = {"Z": "Finish", "A": "Insert card"}
        while True:
            print("-----------------------------------------------------------------------\n")
            operation = input("Enter the capital alphabet to choose which operation to conduct \n" +
                            f"{alphabets} \n")
            print("-----------------------------------------------------------------------\n")

            if operation not in alphabets:
                if language:
                    print("잘못된 operation을 입력하였습니다. 대문자 알파벳을 입력해주세요.")
                else:
                    print("Wrong input for the operation. Please enter the capital alphabet.")

            elif operation == "A":
                is_valid = atm_machine.insert_card(card)
                if not is_valid:
                    break

                del alphabets["A"]
                alphabets["B"] = "Eject card"
                alphabets["C"] = "Enter pin"
            
            elif operation == "B":
                atm_machine.eject_card()
                break

            elif operation == "C":
                is_done = False
                while not is_done:
                    pin = input("Enter pin to access the accounts in card. \n" +
                                "Note that there are only 5 trials. \n" + 
                                "After 5 trials, the card is automatically ejected and no longer useable. \n")
                    is_done = atm_machine.check_pin(pin)
                
                if card.cardID in atm_machine.black_list:
                    if language:
                        print("해당 카드는 더이상 사용할 수 없기 때문에 새로운 카드를 만드는 과정으로 돌아갑니다.")
                    else:
                        print("The card is no longer usable, so we return to the process of creating a new card.")
                    print("-----------------------------------------------------------------------\n")
                    break
                else:
                    del alphabets["C"]
                    alphabets["D"] = "Select account"

            elif operation == "D":
                num_account = len(card.accounts)
                index = input("Enter the account index to query. \n" +
                              f"The account index should be among {list(range(1,num_account+1))}. \n")
                index = int(index)
                atm_machine.select_account(index)
                alphabets["D"] = "Switch account"
                alphabets["E"] = "See balance"
                alphabets["F"] = "Deposit"
                alphabets["G"] = "Withdraw"

            elif operation == "E":
                atm_machine.do_request(task=0)

            elif operation in ["F", "G"]:
                money = input("Enter the amount of money to deposit or withdraw \n")
                money = int(money)
                if operation == "F":
                    atm_machine.do_request(task=1, money=money)
                else:
                    atm_machine.do_request(task=2, money=money)

            elif operation == "Z":
                if language:
                    print("기기 사용을 종료합니다.")
                else:
                    print("The atm machine is turned off.")
                return


In [3]:
main(num_account=3)

Let's create a new atm 
Enter 0 or 1 to choose a language for instruction by atm: 0 for English, 1 for Korean 
 0


English language is chosen.
-----------------------------------------------------------------------



Let's create a new card 
Enter two digits with ',' between: 1st for cardID and 2nd for pin. ex) 1, 2 
  1, 2


-----------------------------------------------------------------------



Let's create a new account 
Enter two digits with ',' between: 1st for accountID and 2nd for balance. ex) 3, 4 
  11, 22
Let's create a new account 
Enter two digits with ',' between: 1st for accountID and 2nd for balance. ex) 3, 4 
  33, 44
Let's create a new account 
Enter two digits with ',' between: 1st for accountID and 2nd for balance. ex) 3, 4 
  55, 66


...

-----------------------------------------------------------------------



Enter the amount of money to deposit or withdraw 
 10


10 won is withdrawed from the 2th account.
The balance of the 2th account is 134.
-----------------------------------------------------------------------



Enter the capital alphabet to choose which operation to conduct 
{'Z': 'Finish', 'B': 'Eject card', 'D': 'Switch account', 'E': 'See balance', 'F': 'Deposit', 'G': 'Withdraw'} 
 Z


-----------------------------------------------------------------------

The atm machine is turned off.
